In [ ]:
import pickle
import numpy as np
import pandas as pd
import strat_models
import matplotlib.pyplot as plt
import networkx as nx

from risk_return_models import *
from utils import *

np.random.seed(0)

In [ ]:
df_train = pd.read_csv("df_train.csv", index_col="Date")
df_val = pd.read_csv("df_test.csv", index_col="Date")
df_holdout = pd.read_csv("df_holdout.csv", index_col="date")

_, num_assets = df_train.shape

Z_train = pd.read_csv("Z_train.csv", index_col="Date")
Z_val = pd.read_csv("Z_test.csv", index_col="Date")
Z_holdout = pd.read_csv("Z_holdout.csv", index_col="Date")

num_quantiles = 10

In [ ]:
train_mean = get_data_dict(df_Y=df_train, df_Z=Z_train, num_assets=num_assets)
val_mean = get_data_dict(df_Y=df_val, df_Z=Z_val, num_assets=num_assets)

In [ ]:
#here, we fit the stratified return model.
kwargs = dict(verbose=True, abs_tol=1e-6, maxiter=2000, rho=10)

M, local, w1, w2, w3, w4 = 0.01, 0.0075, 5, 10, 1000, 1000
                            
print(M, local, w1, w2, w3, w4)

G_vix = nx.path_graph(num_quantiles) #vix quantiles (deciles)
G_unemp = nx.path_graph(num_quantiles) #volume quantiles
G_inflation = nx.path_graph(num_quantiles) #volume quantiles
G_mort = nx.path_graph(num_quantiles) #volume quantiles

strat_models.set_edge_weight(G_vix, w1)
strat_models.set_edge_weight(G_unemp, w2)
strat_models.set_edge_weight(G_inflation, w3)
strat_models.set_edge_weight(G_mort, w4)

G = strat_models.cartesian_product([G_vix, G_unemp, G_inflation, G_mort])

loss = huber_return_loss(M=M)
reg = strat_models.sum_squares_reg(lambd=local)

bm = strat_models.BaseModel(loss=loss,reg=reg)
sm = strat_models.StratifiedModel(BaseModel=bm, graph=G)

sm.fit(data=train_mean, **kwargs)

preds_train = np.vstack([
    sm.G._node[tuple(Z_train.loc[date].values)]["theta"] for date in Z_train.index])

preds_val = np.vstack([
    sm.G._node[tuple(Z_val.loc[date].values)]["theta"] for date in Z_val.index])

sm_corr_train = CORR_SM(preds=preds_train, df=df_train)
sm_corr_val = CORR_SM(preds=preds_val, df=df_val)

print("Stratified return model correlations:")
print("\ttrain = {}".format(sm_corr_train))
print("\tval = {}".format(sm_corr_val))

In [ ]:
#Here, we fit the common model.
common_mean = df_train.mean(axis=0)
    
preds_train = np.vstack([common_mean for date in Z_train.index])
preds_val = np.vstack([common_mean for date in Z_val.index])

corr_train = CORR_SM(preds=preds_train, df=df_train)
corr_val = CORR_SM(preds=preds_val, df=df_val)

print("Common return model correlations:")
print("\t train = {}".format(corr_train))
print("\t val = {}".format(corr_val))

In [ ]:
means = dict()
for node in sm.G.nodes():
    means[node] = sm.G._node[node]["theta"].copy()
    
rets = pd.DataFrame(data=np.vstack([means[key] for key in means.keys()]), columns=df_train.columns)

tab = rets.describe().loc[["50%", "min", "max"]].rename(index={"50%":"median"}).T
tab["common"] = common_mean
tab = tab[["common", "median", "min", "max"]] * 100

print(tab.round(3))

In [ ]:
#putting covariance data in form to be accepted by stratified model.

train_cov = get_data_dict(df_Y=df_train, df_Z=Z_train, num_assets=num_assets)
val_cov = get_data_dict(df_Y=df_val, df_Z=Z_val, num_assets=num_assets)

for i in range(len(train_cov["Y"])):
    if not np.allclose(train_cov["Y"][i], 0):
        train_cov["Y"][i] = (train_cov["Y"][i])*100
for i in range(len(val_cov["Y"])):
    if not np.allclose(val_cov["Y"][i], 0):
        val_cov["Y"][i] = (val_cov["Y"][i])*100

In [ ]:
"""STRATIFIED MODEL"""
#here, we fit the stratified risk model.
kwargs = dict(verbose=True, abs_tol=1e-6, maxiter=2000, rho=10)

w1, w2, w3, w4 = 1, 0.2, 100, 50

G_vix = nx.path_graph(num_quantiles) #vix quantiles (deciles)
G_unemp = nx.path_graph(num_quantiles) #volume quantiles
G_inflation = nx.path_graph(num_quantiles) #volume quantiles
G_mort = nx.path_graph(num_quantiles)

strat_models.set_edge_weight(G_vix, w1)
strat_models.set_edge_weight(G_unemp, w2)
strat_models.set_edge_weight(G_inflation, w3)
strat_models.set_edge_weight(G_mort, w4)

G = strat_models.cartesian_product([G_vix, G_unemp, G_inflation, G_mort])

loss = covariance_max_likelihood_loss()
reg = strat_models.trace_reg(lambd=0)#we include no regularization (lambd=0).

bm = strat_models.BaseModel(loss=loss,reg=reg)
sm = strat_models.StratifiedModel(BaseModel=bm, graph=G)

sm.fit(data=train_cov, **kwargs)

covs = dict() 
for node in sm.G.nodes():
    covs[node] = np.linalg.inv(sm.G._node[node]["theta"].copy()).copy()
    
print("Stratified risk model losses:")
print("\ttrain = {}".format(sm.anll(train_cov)))
print("\tval = {}".format(sm.anll(val_cov)))

In [ ]:
#Common model
#Here, we just put the empirical covariance matrix into a stratified model class
#so we can invoke the ANLL functions to compare the common model to the 
#stratified model.

theta_common = (df_train*100).cov().values

G_vix = nx.path_graph(num_quantiles) #vix quantiles (deciles)
G_unemp = nx.path_graph(num_quantiles) #volume quantiles
G_inflation = nx.path_graph(num_quantiles) #volume quantiles
G_mort = nx.path_graph(num_quantiles)

G = strat_models.cartesian_product([G_vix, G_unemp, G_inflation, G_mort])

loss = covariance_max_likelihood_loss()
reg = strat_models.trace_reg(lambd=local)

bm_common = strat_models.BaseModel(loss=loss,reg=reg)
sm_common = strat_models.StratifiedModel(BaseModel=bm_common, graph=G)

for node in sm.G.nodes():
    sm_common.G._node[node]["theta"] = np.linalg.inv(theta_common)
    sm_common.G._node[node]["theta_tilde"] = np.linalg.inv(theta_common)
    sm_common.G._node[node]["theta_hat"] = np.linalg.inv(theta_common)
    
print("train:", sm_common.anll(train_cov))
print("validation:", sm_common.anll(val_cov))

In [ ]:
common_vols = np.sqrt((100*df_train).cov().values.diagonal()/(100*100))

vols = pd.DataFrame(data=np.vstack([np.sqrt(covs[key].diagonal()/(100*100)) for key in covs.keys()]), 
                    columns=df_train.columns)

tab = vols.describe().loc[["50%", "min", "max"]].rename(index={"50%":"Median"}).T
tab["Common"] = common_vols
tab = tab[["Common", "Median", "min", "max"]]

print((tab*100).round(3))

In [ ]:
assets = df_train.columns
VTI_idx = np.where(assets=="VTI")[0][0]

common_corrs = pd.DataFrame(data=correlation_from_covariance(df_train.cov().values)[VTI_idx].reshape(-1,1),
                            index=assets,
                            columns=["Common"])

corrs_strat = []
for key in covs.keys():
    corr_mtx = correlation_from_covariance(covs[key])
    corrs_strat += [corr_mtx[VTI_idx]]

corrs = pd.DataFrame(data=np.vstack(corrs_strat),
                    columns=df_train.columns)

tab = corrs.describe().loc[["50%", "min", "max"]].rename(index={"50%":"Median"}).T
tab["Common"] = common_corrs

tab = tab[["Common", "Median", "min", "max"]]

print((tab).round(3))

In [ ]:
#For the backtest on the held-out data, 
#we retrain on all of the training and validation data.
#Those models are in these .pkl files.

means_file = open('means.pkl', 'rb')
means = pickle.load(means_file)
means_file.close()

common_means_file = open('common_means.pkl', 'rb')
common_means = pickle.load(common_means_file)
common_means_file.close()

covs_file = open('covs.pkl', 'rb')
covs = pickle.load(covs_file)
covs_file.close()

common_inv_cov_file = open('common_inv_cov.pkl', 'rb')
common_inv_cov = pickle.load(common_inv_cov_file)
common_inv_cov_file.close()

tau = pd.read_csv("tau.csv", index_col="TICKER", squeeze=True)
kappa = pd.read_csv("kappa.csv", index_col="TICKER", squeeze=True)

In [ ]:
portfolio_values, df_Zs, W, sharpes_strat, returns_strat = backtest(returns=df_holdout.values,
                                            Z_returns=Z_holdout,
                                            benchmark=df_holdout["VTI"],
                                            means=means, covs=covs,
                                            lev_lim=2,
                                            bottom_sec_limit=-0.25, 
                                            upper_sec_limit=0.3,
                                            shorting_cost=2.37,
                                            tcost=0.32,
                                            MAXRISK=2.5e-5,
                                            tau=tau.values,
                                            kappa=kappa.values)

portfolio_values_common, Wcommon, sharpes_common = backtest_common(returns=df_holdout.values,
                                            Z_returns=Z_holdout,
                                            benchmark=df_holdout["VTI"],
                                            mean = common_means[(0,0,0,0)],
                                            cov = common_inv_cov,
                                            lev_lim=2,
                                            bottom_sec_limit=-0.25,
                                            upper_sec_limit=0.3,
                                            shorting_cost=1.33,
                                            tcost=0.075,
                                            MAXRISK=2.5e-5,
                                            tau=tau.values,
                                            kappa=kappa.values)

portfolio_values["Common model policy"] = portfolio_values_common["Common model policy"]

print("SHARPE OF STRATIFIED MODEL POLICY:", sharpes_strat["Stratified model policy"])
print("SHARPE OF COMMON MODEL POLICY:", sharpes_common["Common model policy"])

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8), sharex=True)

mapping = {"vix":"Volatility", "unemp":"Unemployment", "inf":"Inflation", "mort":"Mortgage"}
df_Zs = df_Zs.rename(columns=mapping)

(1+df_Zs).plot(ax=ax[0], color=["purple", "orange", "gold", "red"], fontsize="x-large",
              yticks=range(1,11))

portfolio_values = portfolio_values.rename(columns={"benchmark":"VTI"})
portfolio_values.plot(ax=ax[1], color=["black", "blue", "red"], fontsize="x-large")

ax[0].set_xlabel("Date", fontsize="x-large")
ax[0].set_ylabel("Economic conditions decile", fontsize="x-large")
ax[1].set_xlabel("Date", fontsize="x-large")
ax[1].set_ylabel("Portfolio value", fontsize="x-large")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,8), sharex=True)

W[0] = W[0].reshape(-1,1)
Wcommon[0] = Wcommon[0].reshape(-1,1)

pd.DataFrame(np.hstack(W).T[1:,:], index=portfolio_values.index[1:]).plot(ax=ax[0], 
                                                                          fontsize="x-large", 
                                                                          legend=False)
pd.DataFrame(np.hstack(Wcommon).T[1:,:], index=portfolio_values.index[1:]).plot(ax=ax[1], 
                                                                                fontsize="x-large", 
                                                                                legend=False)

ax[0].set_ylabel("Stratified model\nholdings", fontsize="x-large")
ax[0].set_xlabel("Date", fontsize="x-large")
ax[1].set_ylabel("Common model\nholdings", fontsize="x-large")
ax[1].set_xlabel("Date", fontsize="x-large")

plt.tight_layout()
plt.show()

In [ ]:
def annualized_return_risk(portfolio_values):
    
    portfolio_values = portfolio_values.pct_change()
    P = 250

    ann_return = portfolio_values.mean()*P
    ann_risk = portfolio_values.std()*np.sqrt(P)
    
    return ann_return, ann_risk

ret_sm, risk_sm = annualized_return_risk(portfolio_values=portfolio_values["Stratified model policy"])
ret_common, risk_common = annualized_return_risk(portfolio_values=portfolio_values["Common model policy"])
ret_bmark, risk_bmark = annualized_return_risk(portfolio_values=portfolio_values["VTI"])

portfolio_returns = portfolio_values.pct_change().dropna()
alphas = portfolio_returns.subtract(portfolio_returns["VTI"],axis=0)
IR = np.sqrt(250)*alphas.mean() / alphas.std()


print("\t\tANNUALIZED")
print("\t\tRETURN          RISK               SHARPE        IR")
print("Stratified\nmodel:")
print("\t", ret_sm, risk_sm, ret_sm/risk_sm, IR["Stratified model policy"])
print("Common\nmodel:")
print("\t", ret_common, risk_common, ret_common/risk_common, IR["Common model policy"])
print("Benchmark:")
print("\t", ret_bmark, risk_bmark, ret_bmark/risk_bmark, 0)

In [ ]:
fs = ["mktrf", "smb", "hml", "umd"]
factors = pd.read_csv("fama_french_factors.csv", index_col="date")
factors.index = pd.to_datetime(factors.index)
factors["alpha"] = 1
Y = portfolio_values["Stratified model policy"].pct_change().dropna()
Y.index = pd.to_datetime(Y.index)
X = factors.loc[Y.index,fs+["alpha"]]

strat_ff = np.linalg.inv(X.T@X)@X.T@Y
strat_ff = pd.DataFrame(strat_ff.values.reshape(1,-1), columns=fs+["alpha"], index=["Stratified model policy"]).T

factors = pd.read_csv("fama_french_factors.csv", index_col="date")
factors.index = pd.to_datetime(factors.index)
factors["alpha"] = 1
Y = portfolio_values["Common model policy"].pct_change().dropna()
Y.index = pd.to_datetime(Y.index)
X = factors.loc[Y.index,fs+["alpha"]]

common_ff = np.linalg.inv(X.T@X)@X.T@Y
common_ff = pd.DataFrame(common_ff.values.reshape(1,-1), columns=fs+["alpha"], index=["Common model policy"]).T

factors = pd.read_csv("fama_french_factors.csv", index_col="date")
factors.index = pd.to_datetime(factors.index)
factors["alpha"] = 1
Y = portfolio_values["VTI"].pct_change().dropna()
Y.index = pd.to_datetime(Y.index)
X = factors.loc[Y.index,fs+["alpha"]]

bmark_ff = np.linalg.inv(X.T@X)@X.T@Y
bmark_ff = pd.DataFrame(bmark_ff.values.reshape(1,-1), columns=fs+["alpha"], index=["VTI"]).T

ff_table = pd.concat([strat_ff, common_ff, bmark_ff], axis=1)
print(ff_table)